# Implementatie van Lloyds algoritme

Omdat het originele artikel waarin Lloyds algoritme gepubliceerd is niet over
computerwetenschap gaat maar over de elektrotechniek staat er niet echt een
duidelijk stuk pseudocode in het artikel. Om deze reden gebruiken we een ander
artikel waar Lloyds algoritme ook in beschreven wordt, namelijk Kmeans++
<sup>\[[1](#src2007)\]</sup>. Hierin worden de stappen als volgt beschreven:

1. Kies een willekeurig aantal clustercentra.
2. Voor elk clustercentrum, laat alle punten in de dataset wijzen naar het 
dichtstbijzijnde clustercentrum.
3. Verander elk clustercentrum naar de zwaartepunt van alle datapunten die naar
dit clustercentrum wijzen.
4. Herhaal stappen 2 en 3 totdat er niks meer veranderd.

## Stap 1

In [3]:
import numpy as np
import numpy.typing as npt

In [30]:
def choose_random_centers(dataset: npt.ArrayLike, num_clusters: int) -> npt.NDArray:
    return dataset[np.random.choice(dataset.shape[0], num_clusters, replace=False)]

In [32]:
test_set: npt.NDArray = np.array([
    [1, 4, 6, 1],
    [3, 5, 1, 4],
    [1, 5, 0, 1],
    [8, 1, 4, 6],
    [9, 9, 9, 9],
])

choose_random_centers(test_set, 2)

array([[1, 4, 6, 1],
       [8, 1, 4, 6]])

## Stap 2

In [39]:
def get_euclidean_distance[ArrT: npt.ArrayLike](vector_one: ArrT, vector_two: ArrT) -> npt.NDArray:
    return np.linalg.norm(vector_one - vector_two)

In [49]:
print("Zelf berekende Euclidische afstand:\t", np.sqrt((1 - 3)**2 + (4 - 5)**2 + (6 - 1)**2 + (1 - 4)**2))
print("np.linalg.norm:\t\t\t\t", get_euclidean_distance(test_set[0], test_set[1]))

Zelf berekende Euclidische afstand:	 6.244997998398398
np.linalg.norm:				 6.244997998398398


In [51]:
def get_matrix_euclidean_distance[ArrT: npt.ArrayLike](matrix: ArrT, vector: ArrT) -> npt.NDArray:
    return np.array([get_euclidean_distance(arr, vector) for arr in matrix])

In [56]:
get_matrix_euclidean_distance(test_set[1:,:], test_set[0])

array([ 6.244998  ,  6.08276253,  9.32737905, 12.72792206])

In [57]:
def get_closest_cluster_center[ArrT: npt.ArrayLike](matrix: ArrT, centers: ArrT) -> npt.ArrayLike:
    return np.array([np.argmin(get_matrix_euclidean_distance(centers, vector)) for vector in matrix])

In [59]:
test_centers: npt.NDArray = choose_random_centers(test_set, 2)
print(test_centers)

[[3 5 1 4]
 [1 5 0 1]]


In [61]:
closest_centers: npt.NDArray = get_closest_cluster_center(test_set, test_centers)

for i, vector in enumerate(test_set):
    print(vector, "is het dichtstbij:", test_centers[closest_centers[i]])

[1 4 6 1] is het dichtstbij: [1 5 0 1]
[3 5 1 4] is het dichtstbij: [3 5 1 4]
[1 5 0 1] is het dichtstbij: [1 5 0 1]
[8 1 4 6] is het dichtstbij: [3 5 1 4]
[9 9 9 9] is het dichtstbij: [3 5 1 4]


## Stap 3

In [64]:
def get_center_of_mass_for_cluster_center[ArrT: npt.ArrayLike](
    matrix: ArrT, closest_centers: ArrT, cluster_idx: int
) -> npt.NDArray:
    mask: npt.NDArray = closest_centers == cluster_idx
    return np.average(matrix[mask,:], axis=0)

array([6.66666667, 5.        , 4.66666667, 6.33333333])

In [68]:
def get_all_center_of_masses[ArrT: npt.ArrayLike](matrix: ArrT, closest_centers: ArrT, num_clusters: int) -> npt.NDArray:
    return np.array([
        get_center_of_mass_for_cluster_center(matrix, closest_centers, i)
        for i
        in range(num_clusters)
    ])

In [73]:
new_centers: npt.NDArray = get_all_center_of_masses(test_set, closest_centers, 2)

print(new_centers)

[[6.66666667 5.         4.66666667 6.33333333]
 [1.         4.5        3.         1.        ]]


## Stap 4

In [107]:
def run_lloyds(matrix: npt.NDArray, num_clusters: int) -> tuple[npt.NDArray, npt.NDArray]:
    cluster_centers: npt.NDArray = choose_random_centers(matrix, num_clusters)
    closest_centers: npt.NDArray = get_closest_cluster_center(matrix, cluster_centers)

    while not ((new_centers := get_all_center_of_masses(matrix, closest_centers, num_clusters)) == cluster_centers).all():
        cluster_centers = new_centers
        closest_centers = get_closest_cluster_center(matrix, cluster_centers)

    return cluster_centers, closest_centers

In [108]:
run_lloyds(test_set, 2)


lol
lol
lol


(array([[1.66666667, 4.66666667, 2.33333333, 2.        ],
        [8.5       , 5.        , 6.5       , 7.5       ]]),
 array([0, 0, 0, 1, 1]))

## Sources

<a id="src2007"></a> \[1\] Arthur, D., & Vassilvitskii, S. (2007). k-means++: the advantages of careful seeding. Soda, 1027–1035. https://doi.org/10.5555/1283383.1283494